In [12]:
!pip install opencv-python

In [13]:
!pip install keras

In [14]:
!pip install tensorflow-gpu

In [15]:
conda install -c conda-forge google-colab

Solving environment: ...working... done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [16]:
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
import tensorflow as tf
from keras.models import Sequential, Model,load_model
from keras.callbacks import EarlyStopping,ModelCheckpoint
#from google.colab.patches import cv2_imshow
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPool2D,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
import joblib

In [2]:
tf.test.gpu_device_name()

''

In [3]:
train_path=r"C:\Users\kais\Desktop\Kais\Stage\ByColor\train"
test_path=r"C:\Users\kais\Desktop\Kais\Stage\ByColor\test"
class_names=os.listdir(train_path)
class_names_test=os.listdir(test_path)

In [4]:
print(class_names)
print(class_names_test)

['Blanc', 'Bleu', 'Gris', 'Jaune', 'Marron', 'Multicouleurs', 'Noir', 'Rouge', 'Transparent']
['Blanc', 'Bleu', 'Gris', 'Jaune', 'Marron', 'Multicouleurs', 'Noir', 'Rouge', 'Transparent']


In [5]:
### Sample datasets images
image_argent=cv2.imread(r"C:\Users\kais\Desktop\Kais\Stage\ByColor\test\Gris\102935_1599492623824.jpg")
#cv2.imshow('ImageWindow', image_argent)
cv2.waitKey()
image_noir=cv2.imread(r"C:\Users\kais\Desktop\Kais\Stage\ByColor\test\Noir\15653_1599497520179.jpg")
#cv2_imshow(image_noir)

In [6]:
train_datagen =ImageDataGenerator(preprocessing_function=preprocess_input,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  validation_split=0.4)

In [7]:
train_generator = ImageDataGenerator(zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15)
test_datagen = ImageDataGenerator()

In [9]:
IMAGE_SIZE = (224, 224)

In [10]:
train_generator = train_datagen.flow_from_directory(train_path,target_size=IMAGE_SIZE,batch_size=32,shuffle=True,color_mode="rgb",class_mode='categorical',subset='training')
test_generator = train_datagen.flow_from_directory(test_path,target_size=IMAGE_SIZE,batch_size=32,shuffle=False,color_mode="rgb",class_mode='categorical',subset='validation')
valid_generator=train_datagen.flow_from_directory(test_path,target_size=IMAGE_SIZE,batch_size=32,class_mode ='categorical',subset='validation')


Found 4122 images belonging to 9 classes.
Found 186 images belonging to 9 classes.
Found 186 images belonging to 9 classes.


In [11]:
base_model = ResNet50(include_top = False,weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x)
predictions=Dense(train_generator.num_classes,activation='softmax')(x)
model= Model(inputs=base_model.input,outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
    
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_generator,epochs=30)

Epoch 1/30
129/129 [==============================] - 908s 7s/step - loss: 1.2213 - accuracy: 0.6206
Epoch 2/30
129/129 [==============================] - 420s 3s/step - loss: 0.7752 - accuracy: 0.7339
Epoch 3/30
129/129 [==============================] - 468s 4s/step - loss: 0.6981 - accuracy: 0.7613
Epoch 4/30
129/129 [==============================] - 369s 3s/step - loss: 0.6189 - accuracy: 0.7858
Epoch 5/30
129/129 [==============================] - 368s 3s/step - loss: 0.5569 - accuracy: 0.8076
Epoch 6/30
129/129 [==============================] - 370s 3s/step - loss: 0.5022 - accuracy: 0.8234
Epoch 7/30
129/129 [==============================] - 311s 2s/step - loss: 0.4714 - accuracy: 0.8375
Epoch 8/30
129/129 [==============================] - 274s 2s/step - loss: 0.4608 - accuracy: 0.8350
Epoch 9/30
129/129 [==============================] - 174s 1s/step - loss: 0.4032 - accuracy: 0.8622
Epoch 10/30
129/129 [==============================] - 172s 1s/step - loss: 0.3731 - accura

In [13]:
model.save_weights('Models/couleur/finalized_model.h5')

In [ ]:
#model.load_weights('Models/couleur/finalized_model.h5')
#model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
#model.fit(train_generator,epochs=60)

In [17]:
filename = 'Models/couleur/finalized_model.sav'
joblib.dump(model, filename)

INFO:tensorflow:Assets written to: ram://d3bab971-513a-458d-9c23-663de901bb16/assets


INFO:tensorflow:Assets written to: ram://d3bab971-513a-458d-9c23-663de901bb16/assets


['Models/couleur/finalized_model.sav']

In [18]:
model_json = model.to_json()
with open(r'Models/couleur/finalized_model.json',"w") as json_file:
  json_file.write(model_json)

### **COLOR PREDICTION**

In [ ]:
from keras.models import model_from_json

In [ ]:
def predict_(image_path):
    #Load the Model from Json File
    json_file = open(r"C:\Users\kais\Desktop\Kais\Stage\ByColor\model.json', 'r')
    model_json_c = json_file.read()
    json_file.close()
    model_c = model_from_json(model_json_c)
    #Load the weights
    model_c.load_weights(r"C:\Users\kais\Desktop\Kais\Stage\ByColor\best_model.h5")
    #Compile the model
    opt = SGD(lr=1e-4, momentum=0.9)
    model_c.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
    #load the image you want to classify
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224,224))
    cv2_imshow(image)
    #predict the image
    preds = model_c.predict(np.expand_dims(image, axis=0))[0]
    if preds==0:
        print("Predicted Label:Cat")
    else:
        print("Predicted Label: Dog")

In [ ]:
predict_("/content/gdrive/My Drive/datasets/test/Dog/4.jpg")

In [ ]:
predict_("/content/gdrive/My Drive/datasets/test/Cat/10.jpg")

In [ ]:
predict_("/content/gdrive/My Drive/datasets/test/Cat/7.jpg")